# Parameters
in this cell the used variables can be changed 

height and width correspond to the dimensions of the Workspace, in which the Topology Optimisation is going to be performed

log_dir is the direction where the current best Model will be saved to during training

ts is the amount of timesteps the model will be trained for




In [1]:
import numpy as np
import gymnasium as gym

import autograd.numpy as anp

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter

Here the reward for each state of the workspace is the compliance
which should be minimized or above a certain threshold



In [2]:
height = 13
width = 19
log_dir = "log"
ts = 1e6

density = 1e-4


normals = np.zeros((height, width, 2))
x = 0
y = 1
print("normalsX\n" + str(normals[:,:,x]))
print("normalsY\n" + str(normals[:,:,y]))



number_of_nodes = height * width

normalsX
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
normalsY
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

In [3]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d
    
def get_args(normals, forces, density=1e-4):  # Manage the problem setup parameters
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)

def mbb_beam(width=7, height=7, density=1e-4, y=1, x=0, rd=-1):  # textbook beam example
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = -1
    return normals, forces, density

In [4]:
# TODO write the function to get the reward
# It should return the compliance of the structure
# The compliance is the sum of the forces applied to the structure
# multiplied by the displacement of the nodes
# The goal of the 
def rewardFunction(state):
    return 1

In [5]:
class TopOptEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self):

        super().__init__()
        self.design_space = anp.ones((height, width)) * density

    
        self.action_space = gym.spaces.Discrete(number_of_nodes)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(height, width), dtype=np.float64)

        self.n, self.m = self.design_space.shape
        print("self.n:\t" + str(self.n) + "\nself.m:\t" + str(self.m))
        self._actions_to_coordinates={} 
    
        k=0
        for i in range(self.n):
            for j in range(self.m):
                self._actions_to_coordinates[k]=(i,j)
                k+=1
        print(self._actions_to_coordinates)
        self.reward = 0
        self.step_count = 0

    def step(self, action):
        #
        # TODO finisch the step function for the environment	
        # we need to add the reward function and the termination condition
        #
        #
        # action is the index of the node that is going to be changed
        # action is a number between 0 and number_of_nodes

        self.step_count += 1
        


        self.reward = rewardFunction(self.design_space)
        return self.design_space, self.reward, terminated, False, self._getInfo()
    

    def reset(self, seed=None):
        # Reset the environment
        # This function returns the initial state of the environment
        # The initial state should be a 2D array of size (height, width)
        #
        #TODO: add a way to introduce Randomness to the Forces applied in the Design space
        #

        super().reset(seed=seed)
        self.step_count = 0
        self.design_space = anp.ones((height, width)) * density


        return self.design_space, self._getInfo()
    
    def _getInfo(self):
        # This function returns the information about the environment
        # This function is used to monitor the environment
        # The information should be a dictionary
        # The dictionary should contain the following keys:
        # - step_count: the number of steps that have been executed
        # - current Reward: the reward of the current state
        # - design_space: the current state of the environment
        return {"step_count": self.step_count, 
                "current Reward": self.reward,
                "design_space": self.design_space}


    

        


In [6]:
env = TopOptEnv()
check_env(env, warn=True)

self.n:	13
self.m:	19
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (0, 4), 5: (0, 5), 6: (0, 6), 7: (0, 7), 8: (0, 8), 9: (0, 9), 10: (0, 10), 11: (0, 11), 12: (0, 12), 13: (0, 13), 14: (0, 14), 15: (0, 15), 16: (0, 16), 17: (0, 17), 18: (0, 18), 19: (1, 0), 20: (1, 1), 21: (1, 2), 22: (1, 3), 23: (1, 4), 24: (1, 5), 25: (1, 6), 26: (1, 7), 27: (1, 8), 28: (1, 9), 29: (1, 10), 30: (1, 11), 31: (1, 12), 32: (1, 13), 33: (1, 14), 34: (1, 15), 35: (1, 16), 36: (1, 17), 37: (1, 18), 38: (2, 0), 39: (2, 1), 40: (2, 2), 41: (2, 3), 42: (2, 4), 43: (2, 5), 44: (2, 6), 45: (2, 7), 46: (2, 8), 47: (2, 9), 48: (2, 10), 49: (2, 11), 50: (2, 12), 51: (2, 13), 52: (2, 14), 53: (2, 15), 54: (2, 16), 55: (2, 17), 56: (2, 18), 57: (3, 0), 58: (3, 1), 59: (3, 2), 60: (3, 3), 61: (3, 4), 62: (3, 5), 63: (3, 6), 64: (3, 7), 65: (3, 8), 66: (3, 9), 67: (3, 10), 68: (3, 11), 69: (3, 12), 70: (3, 13), 71: (3, 14), 72: (3, 15), 73: (3, 16), 74: (3, 17), 75: (3, 18), 76: (4, 0), 77: (4, 1), 78: (4, 2), 79:

c:\Users\adria\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\env_checker.py:263: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


NameError: name 'terminated' is not defined